In [4]:
import urllib.request
import urllib.parse
import encodings.idna


url = 'http://localhost:8000'
f = urllib.request.urlopen(url)
print(f.read().decode('utf-8'))

{"Hello":"World"}


In [7]:
import unicodedata
unicodedata.__file__

'C:\\Users\\thiba\\AppData\\Local\\Programs\\Python\\Python39\\DLLs\\unicodedata.pyd'

In [9]:
import requests
print(requests.__file__)
requests.get('https://api.github.com')

C:\Users\thiba\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py


<Response [200]>